In [11]:
import numpy as np
import pandas as pd
from collections import defaultdict
import operator
import time, datetime
from utils import *

In [3]:
bath_path = '/mnt/qb/work/mlcolab/hzhou52/kt/assistment/2012-2013-data-with-predictions-4-final.csv'
log = pd.read_csv(bath_path, encoding = "utf-8",low_memory=False)

In [5]:
log.columns

Index(['problem_log_id', 'skill', 'problem_id', 'user_id', 'assignment_id',
       'assistment_id', 'start_time', 'end_time', 'problem_type', 'original',
       'correct', 'bottom_hint', 'hint_count', 'actions', 'attempt_count',
       'ms_first_response', 'tutor_mode', 'sequence_id', 'student_class_id',
       'position', 'type', 'base_sequence_id', 'skill_id', 'teacher_id',
       'school_id', 'overlap_time', 'template_id', 'answer_id', 'answer_text',
       'first_action', 'problemlogid', 'Average_confidence(FRUSTRATED)',
       'Average_confidence(CONFUSED)', 'Average_confidence(CONCENTRATING)',
       'Average_confidence(BORED)'],
      dtype='object')

In [6]:
check_nan(log)
count_unique(log, ['user_id', 'skill_id', 'problem_id'])
print(len(log))

number of NaN in column problem_log_id: 0
number of NaN in column skill: 3493190
number of NaN in column problem_id: 0
number of NaN in column user_id: 0
number of NaN in column assignment_id: 0
number of NaN in column assistment_id: 0
number of NaN in column start_time: 0
number of NaN in column end_time: 0
number of NaN in column problem_type: 0
number of NaN in column original: 0
number of NaN in column correct: 0
number of NaN in column bottom_hint: 60348
number of NaN in column hint_count: 0
number of NaN in column actions: 0
number of NaN in column attempt_count: 0
number of NaN in column ms_first_response: 0
number of NaN in column tutor_mode: 0
number of NaN in column sequence_id: 0
number of NaN in column student_class_id: 0
number of NaN in column position: 0
number of NaN in column type: 0
number of NaN in column base_sequence_id: 0
number of NaN in column skill_id: 3411457
number of NaN in column teacher_id: 0
number of NaN in column school_id: 157
number of NaN in column o

In [7]:
log = log[~log.skill_id.isna()]
log = remove_log(log, 'user_id')
print(len(log))

2710913


# pre-process

In [10]:
# create new data
df = pd.DataFrame(columns=['user_id', 'problem_id', 'skill_id', 'timestamp', 'dwell_time', 'correct'])
df.user_id = log.user_id
df.problem_id = log.problem_id
df.skill_id = log.skill_id
# df.timestamp = log.start_time.apply(lambda t: time.mktime(time.strptime(t, '%Y-%m-%d %H:%M:%S')))
# df.dwell_time = df.timestamp - log.end_time.apply(lambda t: time.mktime(time.strptime(t, '%Y-%m-%d %H:%M:%S')))
df.correct = log.correct * 1

time_format = "%Y-%m-%d %H:%M:%S"
log.startime = log.start_time.apply(lambda t: time.mktime(datetime.strptime(t.split('.')[0], time_format).timetuple()))
log.endtime = log.end_time.apply(lambda t: time.mktime(datetime.strptime(t.split('.')[0], time_format).timetuple()))
df.timestamp = (log.startime+log.endtime)/2
df.dwell_time = log.endtime - log.startime

In [18]:
# re-index
# - skill
skill_id = list(df['skill_id'].unique())
skill_dict = dict(zip(skill_id, range(len(skill_id))))
df['skill_id'] = df['skill_id'].apply(lambda x: skill_dict[x])

# - problem
pro_id = list(df['problem_id'].unique())
pro_dict = dict(zip(pro_id, range(len(pro_id))))
df['problem_id'] = df['problem_id'].apply(lambda x: pro_dict[x])

# - student
stu_id = list(df['user_id'].unique())
stu_dict = dict(zip(stu_id, range(len(stu_id))))
df['user_id'] = df['user_id'].apply(lambda x: stu_dict[x])

In [19]:
df

,user_id,problem_id,skill_id,timestamp,dwell_time,correct
1,0,0,0,1.349773e+09,21.0,1.0
2,0,1,1,1.362650e+09,8.0,0.0
3,0,2,2,1.377021e+09,25.0,1.0
5,1,3,3,1.355342e+09,12.0,1.0
7,2,4,4,1.350376e+09,61.0,1.0
...,...,...,...,...,...,...
6123259,18202,2992,47,1.377775e+09,25.0,1.0
6123260,3953,4055,13,1.377786e+09,32.0,1.0
6123262,14539,3945,35,1.377955e+09,7.0,1.0
6123264,18493,3704,19,1.377901e+09,98.0,0.0


In [20]:
df = df.astype({
    'timestamp': np.float64,
    'dwell_time': np.float64,
    'correct': np.float64,
    'problem_id': np.int64,
    'skill_id': np.int64,
    'user_id': np.int64
    })

In [18]:
# Save
df.to_csv('/mnt/qb/work/mlcolab/hzhou52/kt/assistment/12_13/'+'interactions.csv', sep='\t', index=False)

In [21]:
count_unique(df, list(df.columns))
len(df)

number of unique value in user_id: 28118
number of unique value in problem_id: 53084
number of unique value in skill_id: 265
number of unique value in timestamp: 2524471
number of unique value in dwell_time: 26708
number of unique value in correct: 8


2710913